In [80]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

# Define Functions

In [81]:
def dict_from_multilevel_idx(df):
#     Pass a dataframe without multilevel index, return dataframe able to be used by the permile deal
#     expects single level indices
    df_g = df.set_index(['make','model','year'])
    all_makes = df_g.index.remove_unused_levels().levels[0].values
    d = {}
    for make in all_makes:
        subset_mdls = df_g.loc[make].index.remove_unused_levels().levels[0].values
        d[make] = dict(zip(subset_mdls,np.tile(np.array([]),(len(subset_mdls),1))))
        for mdl in subset_mdls:
            d[make][mdl] = df_g.loc[(make,mdl),].index.values
    return d

In [82]:
def save_depreciation_per_year(df,d):    
#     Save images of the depreciation/year
    df = df.set_index(['make','model'])
    makes = list(d.keys())
    for make in makes:
        mdls = list(d[make].keys())
        for mdl in mdls:
            plt.scatter(df.loc[(make,mdl),'year'],df.loc[(make,mdl),'dollars_per_mile'])
            ax = plt.gca()
            ax.set_title(make + ' ' + mdl)
            ax.set_xlim(2004, 2017.5)
            ax.set_ylim(.02, 0.6)
            plt.savefig('/Users/ahakso/Downloads/temp/{}{}'.format(make,mdl))
            plt.clf()

In [83]:
def estimate_recent_years(subframe):
#     New cars often don't have enough data for a good extrapolation. Estimate it from previous years
    if 2016 in subframe.year.values:
        val2016_original = subframe.loc[subframe.loc[:,'year']==2016,'dollars_per_mile'].values[0]
        val_previous = subframe.loc[:,'dollars_per_mile'].iloc[np.nonzero(subframe.loc[:,'year']==2016)[0][0]-1]
        if val2016_original < val_previous:
            subframe.loc[subframe.loc[:,'year']==2016,'dollars_per_mile'] = val_previous*1.05
    if 2017 in subframe.year.values:
        idx_recent_years = subframe.year.isin(range(2011,2017))
        if sum(idx_recent_years)>=4:
            x = subframe.year.loc[idx_recent_years]
            y = subframe.dollars_per_mile.loc[idx_recent_years]
        else:
            x = subframe.year
            y = subframe.dollars_per_mile
        p = np.polyfit(x,y,1)
        val = p[0]*2017+p[1]
        val2017_original = subframe.loc[subframe.loc[:,'year']==2017,'dollars_per_mile'].values[0]
        val_previous = subframe.loc[:,'dollars_per_mile'].iloc[np.nonzero(subframe.loc[:,'year']==2017)[0][0]-1]
#         print('original: {}\ncurrent: {}\nprevious: {}\n'.format(val2017_original,val,val_previous))
        if val < val_previous:
            val = val_previous*1.1
#             print('Updated to: {}\n\n\n'.format(val))
        subframe.loc[subframe.loc[:,'year']==2017,'dollars_per_mile'] = val
#         subframe = subframe.drop(['make','model'],axis=1)    
    return subframe



# Import the combined frame

In [84]:
with open('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/combined_frame.pkl','rb') as f:
    df = pickle.load(f)    

# Fill in random missing vehicle classes

In [85]:
makes = ['Mazda','Mazda','Mazda','Mazda','Nissan','Pontiac','FIAT','Ford','Ford','Mercedes-Benz','Mercedes-Benz']
models = ['Mazda2','Mazda3','Mazda5','Mazda6','Titan','GrandAM','500','F250','F350','CClass','EClass']
vclasses = ['Compact Cars','Compact Cars','Minivan - 2WD','Midsize Cars','Standard Pickup Trucks','Compact Cars',\
            'Minicompact Cars','Standard Pickup Trucks','Standard Pickup Trucks',]
for make,model,vclass in zip(makes,models,vclasses):
    df.loc[(df.loc[:,'make'] == make) & (df.loc[:,'model'] == model),'vclass'] = vclass

# Make adjustments to model names

In [86]:
def manual_make_model(make,model_old,model_new):
#     print('{} models match {} {}'.format(sum((df.make==make) &(df.model==model_old)),make,model_old))
    df.loc[(df.make==make) & (df.model==model_old),'model'] = model_new

manual_make_model('Infiniti','M35x','M35')
manual_make_model('Infiniti','M37x','M37')
manual_make_model('Infiniti','Q70L','Q70')
manual_make_model('Jeep','Grand','Grand Cherokee')
manual_make_model('Mercury','Grand','Grand Marquis')
manual_make_model('Pontiac','Grand','Grand AM')
manual_make_model('Pontiac','GrandAM','Grand AM')
manual_make_model('Suzuki','Grand','Grand Vitara')
manual_make_model('Cadillac','AT','ATS')
manual_make_model('Honda','RidgelineL','Ridgeline')
manual_make_model('Honda','RidgelineL','Ridgeline')
manual_make_model('Honda','RidgelineL-E','Ridgeline')
manual_make_model('Honda','RidgelineL-T','Ridgeline')
manual_make_model('Honda','AccordP','Accord')
manual_make_model('Honda','AccordS','Accord')
manual_make_model('Chevrolet','S','S-10')
manual_make_model('Ford','Fiestaedan','Fiesta')
manual_make_model('Ford','FiestaE','Fiesta')
manual_make_model('Kia','OptimaS','Optima')
manual_make_model('Chrysler','PTCruiser','PT Cruiser')
manual_make_model('Chrysler','TownandCountry','Town and Country')
manual_make_model('Dodge','GrandCaravan','Grand Caravan')
manual_make_model('Ford','FusionHybrid','Fusion Hybrid')
manual_make_model('Ford','Five','Five Hundred')
manual_make_model('Ford','Fl','Flex')
manual_make_model('Dodge','DartGT','Dart GT')
manual_make_model('Hyundai','SantaFe','Santa Fe')
manual_make_model('Jeep','GrandCherokee','Grand Cherokee')
manual_make_model('Lincoln','MKZHybrid','MKZ Hybrid')
manual_make_model('Tesla','Model','Model S')
manual_make_model('Mercedes-Benz','CClass','C-Class')
manual_make_model('Mercedes-Benz','SLClass','SL-Class')
manual_make_model('Mercedes-Benz','Class','E-Class')
manual_make_model('Mercedes-Benz','MClass','M-Class')
manual_make_model('Mercedes-Benz','SClass','S-Class')
manual_make_model('Mercedes-Benz','EClass','E-Class')
for i in range(2,8):
    manual_make_model('BMW','{}'.format(str(i)),'{} Series'.format(str(i)))
    manual_make_model('Mazda','Mazda{}'.format(str(i)),'Mazda {}'.format(str(i)))
for i in range(1,4):    
    manual_make_model('Ford','F{}50'.format(i),'F-{}50'.format(i))
dfmodel_back = df.model       

In [87]:
df.vclass.unique()

array(['Compact Cars', 'Sport Utility Vehicle',
       'Small Sport Utility Vehicle', 'Midsize Cars', 'Subcompact Cars',
       'Large Cars', 'Standard Sport Utility Vehicle',
       'Small Station Wagons', 'Two Seaters', 'Minicompact Cars',
       'Special Purpose Vehicle 2WD', 'Standard Pickup Trucks',
       'Small Pickup Trucks 2WD', 'Vans, Passenger Type',
       'Vans, Cargo Type', 'Minivan - 2WD', nan,
       'Special Purpose Vehicle 4WD', 'Midsize Station Wagons',
       'Small Station Wagon'], dtype=object)

In [88]:
makes = ['FIAT','Ford','Ford','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz']
models = ['500','F-250','F-350','C-Class','E-Class','M-Class','S-Class','SL-Class']
vclasses = ['Minicompact Cars','Standard Pickup Trucks','Standard Pickup Trucks','Compact Cars','Midsize Cars',\
           'Sport Utility Vehicle','Large Cars','Two Seaters']
for make,model,vclass in zip(makes,models,vclasses):
    df.loc[(df.loc[:,'make'] == make) & (df.loc[:,'model'] == model),'vclass'] = vclass

# Manual fill of some key missing mpg, weight, msrp and all seats

In [89]:
def fill_val(df,make,mdl,param,val):
    df.loc[(df.loc[:, 'make']==make) & (df.loc[:, 'model'] == mdl),param] = val
# mpg    
param = 'mpg'
makes = ['Ford','Ford','Ford','Ram','Ram','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz']
models = ['F-150','F-250','F-350','2500','3500','E-Class','M-Class','S-Class','SL-Class']
vals = [23, 15, 13,15,13,25,24,24,23]
for i in range(len(makes)):
    fill_val(df,makes[i],models[i],param,vals[i])

# Seats
param = 'seats'
makes = ['Chrysler','Dodge','Honda','Honda','Mazda','Mercedes-Benz']
models = ['Town and Country','Grand Caravan','Odyssey','Pilot','CX9','SL-Class']
vals = [8,8,8,7,7,2]
for i in range(len(makes)):
    fill_val(df,makes[i],models[i],param,vals[i])
idx_seats = df.seats.isnull()
df.loc[idx_seats,'seats'] = 5

# msrp
param = 'msrp'
makes = ['Chevrolet','Chrysler','Chrysler','Dodge','FIAT','Ford','Ford','Ford','Ford',\
         'Honda','Hyundai','Jeep','MINI','Mazda','Mazda','Mazda','Mercedes-Benz','Mercedes-Benz','Mercedes-Benz',\
         'Mercedes-Benz','Scion','Mercedes-Benz']
models = ['Colorado','PT Cruiser','Town and Country','Grand Caravan','500','F-150','F-250','F-350','Fusion Hybrid',\
          'CRV','Santa Fe','Grand Cherokee','Cooper','CX7','CX9','MX5','C-Class','E-Class','M-Class','SL-Class',\
          'tC','S-Class']
          
vals = [20200, 18500, 22500, 22500, 17000, 27700,33500,35000,28000,24250,31000,31000,22000,22000,33000,26000,41000,\
        53000,60000,88000,20000,93000]
for i in range(len(makes)):
    fill_val(df,makes[i],models[i],param,vals[i])

# weight
param = 'weight'
makes = ['Chevrolet','Chrysler','Chrysler','Dodge','FIAT','Ford','Ford','Ford','Ford',\
         'Honda','Hyundai','Jeep','MINI','Mazda','Mazda','Mazda','Mercedes-Benz','Mercedes-Benz',\
         'Mercedes-Benz','Mercedes-Benz','Nissan','Scion','Hyundai','Lincoln','Mercedes-Benz']
models = ['Colorado','PT Cruiser','Town and Country','Grand Caravan','500','F-150','F-250','F-350','Fusion Hybrid',\
          'CRV','Santa Fe','Grand Cherokee','Cooper','CX7','CX9','MX5','C-Class','E-Class','M-Class','SL-Class',\
          'Titan','tC','Tucson','Navigator','S-Class']
vals = [4200, 3165, 4652,3900,2400,4300,6100, 6500, 3700, 3400, 4100, 4800, 2800, 3700, 4200, 2350, 3800, 4000,\
        4800, 3900, 5500, 3100,3450,5750,4550]
for i in range(len(makes)):
    fill_val(df,makes[i],models[i],param,vals[i])    


# Merge and Manage the c-class and c issue

In [90]:
benzidx = (df.make=='Mercedes-Benz')
combine_params = ('dollars_per_mile','weight','msrp','seats','vclass','mpg','volume')
for year in range(2004,2018):
    cclassidx = (df.model == 'C-Class') & (df.year == year) & benzidx
    cidx = (df.model == 'C') & (df.year == year) & benzidx
    for param in combine_params:
        if df.loc[cclassidx,param].isnull().all() & ~df.loc[cidx,param].isnull().all():
            df.loc[cclassidx,param] = df.loc[cidx,param].values

In [91]:
allcidx = (df.model == 'C') & benzidx
df = df.loc[~allcidx,:]

# Remove models without a null value in the fields required for nearest neighbors

In [92]:
idx = ~df.mpg.isnull() & ~df.msrp.isnull() & ~df.weight.isnull() & ~df.seats.isnull() & ~df.vclass.isnull()
df = df.loc[idx,:]

# Get rid of models with less than 3 years available
Make and exception if the two available years are 2016 and 2017

In [93]:
df = df.groupby(['make','model']).filter(lambda x: x.shape[0]>=3)

In [94]:
df_back = df.copy()

# Estimate  Depreciation Data of recent years

In [95]:
df = df_back.copy()

In [96]:
df = df.groupby(['make','model']).apply(estimate_recent_years)

# Get rid of the ~10 relatively new cars with depreciation values below \$0.02/mi


In [97]:
df = df.drop(df.loc[(df.dollars_per_mile<0.02) & (df.year>= 2016)].index)
df = df.drop(df.loc[(df.dollars_per_mile<0.02) & (df.year>= 2010) & (df.msrp > 15000)].index)


# Get objects to be dumped in final form

In [98]:
d = dict_from_multilevel_idx(df.reset_index())

In [99]:
df = df.set_index(['make','model','year'])

In [100]:
df.loc[('Mercedes-Benz'),:]

count  dollars_per_mile  price_mile_resid        mpg  \
model    year                                                         
C-Class  2006     94          0.032704        813.566926  18.238095   
         2007    163          0.044732        977.130704  18.090909   
         2008    329          0.052620       1150.612046  18.733333   
         2009    349          0.085906       4032.418245  17.300000   
         2012    589          0.078986       1557.206443  20.184211   
         2013    878          0.089463       1966.333856  20.386364   
         2014   1186          0.102075       2308.838310  20.500000   
         2015   2026          0.122129       3212.718137  21.439024   
CLA      2014   1300          0.108837       3469.919952  41.750000   
         2015    317          0.107322       1963.319872  38.800000   
         2016    240          0.208579       2393.718906  27.500000   
         2017    215          0.461875       2440.695428  27.250000   
CLK      2004     42          0.061342       1770.681578  17.666667   
         2005     43          0.063793       1349.201159  18.333333   
         2006     65          0.053479       1709.481861  18.400000   
         2007     65          0.080026       3025.207341  17.600000   
         2008     59          0.077164       1986.836053  17.000000   
         2009     55          0.112036       2914.361398  18.500000   
E-Class  2004     90          0.027108       1201.238700  25.000000   
         2005     79          0.033248       1305.184890  25.000000   
         2006    142          0.033352       1517.903214  25.000000   
         2007    183          0.049067       2746.136296  25.000000   
         2008    224          0.054611       1825.621013  25.000000   
         2009    170          0.080014       2892.714413  25.000000   
         2010    564          0.080346       2096.737846  25.000000   
         2011    715          0.108584       2488.090652  25.000000   
         2012    335          0.127806       2599.962453  25.000000   
         2013    406          0.145988       3107.289765  25.000000   
         2014   1743          0.199372       4254.429406  25.000000   
         2015   1119          0.182575       3783.808395  25.000000   
...              ...               ...               ...        ...   
M-Class  2011    274          0.095880       1546.399464  24.000000   
         2012    296          0.107596       1605.186891  24.000000   
         2013    231          0.145914       2217.959311  24.000000   
         2014    747          0.176195       2771.519765  24.000000   
         2015    949          0.194127       3019.720018  24.000000   
S-Class  2005     42          0.054651       1858.525979  24.000000   
         2006     44          0.062550       2039.011414  24.000000   
         2007    255          0.090369       2031.981580  24.000000   
         2008    233          0.140229       3437.388208  24.000000   
         2009    115          0.163046       3713.805445  24.000000   
         2010    148          0.138496       3575.050735  24.000000   
         2011    120          0.187480       3769.295608  24.000000   
         2012    133          0.180854       2169.884995  24.000000   
         2013    141          0.228748       2603.503995  24.000000   
         2014    239          0.258384       3477.523810  24.000000   
         2015    703          0.374337       6759.105960  24.000000   
         2016     97          0.393054      15286.509407  24.000000   
SL-Class 2004     62          0.113312       2203.921583  23.000000   
         2005     58          0.095579       1997.271568  23.000000   
         2006     45          0.097503       2123.240179  23.000000   
         2007     74          0.122954       2822.408184  23.000000   
         2009     73          0.175176       4253.618955  23.000000   
         2013     87          0.282405       3268.078039  23.000000   
         2014    119          0.292249 

# Dump Final Files

In [61]:
with open('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/combined_frame_and_dict_final.pkl','wb') as f:
    pickle.dump([df,d],f)

In [64]:
# with pd.option_context('display.max_rows', 3000):
#     df.groupby(['make','model']).count()

year  count  dollars_per_mile  \
make          model                                             
Acura         ILX                  5      5                 5   
              MDX                 14     14                14   
              RDX                 10     10                10   
              TL                  11     11                11   
              TLX                  3      3                 3   
              TSX                  9      9                 9   
Audi          A4                  13     13                13   
              A5                   7      7                 7   
              A6                   8      8                 8   
              A7                   4      4                 4   
              A8                   5      5                 5   
              Q3                   3      3                 3   
              Q5                   9      9                 9   
              Q7                   7      7                 7   
              S4                   4      4                 4   
              S5                   6      6                 6   
              allroad              3      3                 3   
BMW           2 Series             4      4                 4   
              3 Series            14     14                14   
              4 Series             4      4                 4   
              5 Series            13     13                13   
              6 Series             8      8                 8   
              7 Series            10     10                10   
              X1                   5      5                 5   
              X3                  11     11                11   
              X5                  13     13                13   
              X6                   3      3                 3   
              Z4                   4      4                 4   
Buick         Enclave             10     10                10   
              Encore               5      5                 5   
              LaCrosse            13     13                13   
              Lucerne              6      6                 6   
              Regal                7      7                 7   
              Verano               6      6                 6   
Cadillac      ATS                  5      5                 5   
              CTS                 14     14                14   
              DTS                  6      6                 6   
              Escalade            14     14                14   
              SRX                 10     10                10   
              STS                  4      4                 4   
              XTS                  5      5                 5   
Chevrolet     Avalanche           10     10                10   
              Aveo                 5      5                 5   
              Camaro               8      8                 8   
              Captiva              4      4                 4   
              Cobalt               6      6                 6   
              Colorado            12     12                12   
              Corvette            14     14                14   
              Cruze                6      6                 6   
              Equinox             13     13                13   
              Express             12     12                12   
              HHR                  6      6                 6   
              Impala              14     14                14   
              Malibu              14     14                14   
              Silverado           11     11                11   
              Sonic                6      6                 6   
              Spark                5      5                 5   
              Suburban            14     14                14   
              Tahoe               14     14                14   
              TrailBlazer          5      5                 5   
  

In [60]:
# save_depreciation_per_year(df.reset_index(),d) 

<Figure size 432x288 with 0 Axes>